In [2]:
from IPython import display
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import colormaps as cmaps
# import pandas as pd
from matplotlib.colors import LogNorm
# import gif
import time
import seaborn as sns

/software/python-anaconda-2022.05-el8-x86_64/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
## File Input for MMR data
direc = r'/project2/abbot/jeffyang/CARMA_mix/'
cases = ["referCase-mix"]



NFILE = len(cases)
#restart = [1]*NFILE
dt = [300]*NFILE
nt = [560]*NFILE
NT = max(nt)

NBIN = 50
NZ = 100
NELEM = 6

[NGROUP, NELEM, NBIN, NGAS, NZ] = [4,6,50,1,100]
itime = np.zeros((NFILE,NT),float)

mmr_all = np.zeros((NFILE,NT,NELEM,NZ,NBIN),float)
vr_all = np.zeros((NFILE,NT,NELEM,NZ,NBIN),float)
conc_all = np.zeros((NFILE,NT,NELEM,NZ,NBIN),float)
ppmv_all = np.zeros((NFILE,NT,NELEM,NZ,NBIN),float)
ppm_all = np.zeros((NFILE,NT,NELEM,NZ,NBIN),float)
mmr_gas_all = np.zeros((NFILE,NT,NGAS,NZ),float)
satice_all = np.zeros((NFILE,NT,NGAS,NZ),float)
satliq_all = np.zeros((NFILE,NT,NGAS,NZ),float)

## Particle data
r = np.zeros((NFILE,NGROUP,NBIN),float)
rmass = np.zeros((NFILE,NGROUP,NBIN),float)

## Initial data
mmr_ini = np.zeros((NFILE,NELEM,NZ,NBIN),float)
ppm_ini = np.zeros((NFILE,NELEM,NZ,NBIN),float)
vr_ini = np.zeros((NFILE,NELEM,NZ,NBIN),float)
conc_ini = np.zeros((NFILE,NELEM,NZ,NBIN),float)
ppmv_ini = np.zeros((NFILE,NELEM,NZ,NBIN),float)
mmr_gas_ini = np.zeros((NFILE,NGAS,NZ),float)
satice_ini = np.zeros((NFILE,NGAS,NZ),float)
satliq_ini = np.zeros((NFILE,NGAS,NZ),float)

z = np.zeros((NFILE,NZ),float)
zkm = np.zeros((NFILE,NZ),float)
p = np.zeros((NFILE,NZ),float)
temp = np.zeros((NFILE,NZ),float)

groups = [0,1,2,2,3,3]
rho = [2,2,0.9,0.9,1,1]
[NGROUP, NELEM, NBIN, NGAS, NZ] = [4,6,50,1,100]


for ifile in range(NFILE):
    print(ifile)
    f1 = open(direc + cases[ifile] + r'/run/carma/carma_nuc_grow.txt', 'r')
#     ftp = open(direc + cases[0] + r'/run/carma-init/carma_nuc_grow.txt', 'r')
    ftp = f1

    ## Read Case Info: NGROUP, NELEM, NBIN, NGAS
    infoLine = ftp.readline()

    ## Read and draw background info
    for iz in range(0,NZ):
        zInfo = ftp.readline()
        zInfo = zInfo.strip().split()
        z[ifile,iz] = zInfo[1]
        zkm[ifile,iz] = z[ifile,iz]/1000.
        p[ifile,iz] = zInfo[2]
        temp[ifile,iz] = zInfo[3]

    ## Read Bin Info
    for igroup in range(0,NGROUP):
        for ibin in range(0,NBIN):
            rInfo = ftp.readline()
            rInfo = rInfo.strip().split()
            r[ifile,igroup,ibin] = float(rInfo[2])  # Unit: um; if / 1e4, convert to cm
            rmass[ifile,igroup,ibin] = float(rInfo[3])

    ## Read Initial Conditions
    itime_ini = float(ftp.readline().strip())
    for iz in range(0, NZ):
        for ielem in range(0, NELEM):
            for ibin in range(0, NBIN):
                mmrInfo = ftp.readline()
                mmrInfo = mmrInfo.strip().split()
                mmr_ini[ifile,ielem,iz,ibin] = float(mmrInfo[3])
                vr_ini[ifile,ielem,iz,ibin] = mmr_ini[ifile,ielem,iz,ibin] * p[ifile,iz] /temp[ifile,iz] /297000./rho[ielem]
                conc_ini[ifile,ielem,iz,ibin] = vr_ini[ifile,ielem,iz,ibin] / (4/3 * np.pi *(r[ifile,groups[ielem],ibin]/1e4)**3)
                ppm_ini[ifile,ielem,iz,ibin] = mmr_ini[ifile,ielem,iz,ibin] * 1e6
                ppmv_ini[ifile,ielem,iz,ibin] = vr_ini[ifile,ielem,iz,ibin] * 1e6

    for iz in range (0, NZ):
        for igas in range(0, NGAS):
            mmrgasInfo = ftp.readline()
            mmrgasInfo = mmrgasInfo.strip().split()
            mmr_gas_ini[ifile,igas,iz] = float(mmrgasInfo[2])
            satliq_ini[ifile,igas,iz] = 1.
            satice_ini[ifile,igas,iz] = 1.

                
    ## Read
    for iit in range(nt[ifile]):
    # Read Time Steps
        timeInfo = f1.readline()
        timeInfo = timeInfo.strip()
        itime[ifile,iit] = float(timeInfo)

        for iz in range(0, NZ):
            for ielem in range(0, NELEM):
                for ibin in range(0, NBIN):
                    mmrInfo = f1.readline()
                    mmrInfo = mmrInfo.strip().split()
                    mmr_all[ifile,iit,ielem,iz,ibin] = float(mmrInfo[3])
                    vr_all[ifile,iit,ielem,iz,ibin] = mmr_all[ifile,iit,ielem,iz,ibin] * p[ifile,iz] /temp[ifile,iz] /297000./rho[ielem]
                    conc_all[ifile,iit,ielem,iz,ibin] = vr_all[ifile,iit,ielem,iz,ibin] / (4/3 * np.pi *(r[ifile,groups[ielem],ibin]/1e4)**3)
                    ppm_all[ifile,iit,ielem,iz,ibin] = mmr_all[ifile,iit,ielem,iz,ibin] * 1e6
                    ppmv_all[ifile,iit,ielem,iz,ibin] = vr_all[ifile,iit,ielem,iz,ibin] * 1e6

        for iz in range(0, NZ):
            for igas in range(0, NGAS):
                mmrgasInfo = f1.readline()
                mmrgasInfo = mmrgasInfo.strip().split()
                mmr_gas_all[ifile,iit,igas,iz] = float(mmrgasInfo[2])
                satliq_all[ifile,iit,igas,iz] = float(mmrgasInfo[3])
                satice_all[ifile,iit,igas,iz] = float(mmrgasInfo[4])

    print("This is a 1-D simulation with:")
    print(str(NGROUP)+" groups, "+ \
      str(NELEM)+" elements, "+ \
      str(NBIN)+" bins, "+ \
      str(NGAS)+" gases, and "+ \
      str(NZ)+" levels.")
    print("The maximum volume mix ratio is: " + str(np.max(ppmv_all[ifile,:,:,:,:]))+" ppmv.")

    f1.close()
    ftp.close()


### CALCULATE WATER PATH
ICWP_cld = np.zeros(([NFILE,NT]),float)
LCWP_cld = np.zeros(([NFILE,NT]),float)

for ifi in range(NFILE):
    for it in range(NT):
        for iz in range(NZ):
            for ibin in range(NBIN):
                ICWP_cld[ifi,it] = ICWP_cld[ifi,it] + conc_all[ifi,it,2,iz,ibin] * rmass[ifi,2,ibin] * (2.0e6/NZ)/(1e-4)
                LCWP_cld[ifi,it] = LCWP_cld[ifi,it] + conc_all[ifi,it,4,iz,ibin] * rmass[ifi,3,ibin] * (2.0e6/NZ)/(1e-4)


0
This is a 1-D simulation with:
4 groups, 6 elements, 50 bins, 1 gases, and 100 levels.
The maximum volume mix ratio is: 0.1667756351392715 ppmv.


In [4]:
### READ RADIATIVE INFO
print("Reading carma_nuc_grow_mie.txt")
[NGROUP, NWAVE, NBIN] = [4,50,50]
wave = np.ndarray((NWAVE),float)
r_refidx_ice = np.ndarray((NGROUP,NWAVE),float)
i_refidx_ice = np.ndarray((NGROUP,NWAVE),float)
r_refidx_water = np.ndarray((NGROUP,NWAVE),float)
i_refidx_water = np.ndarray((NGROUP,NWAVE),float)
qext = np.ndarray((NGROUP,NWAVE,NBIN),float)
ssa = np.ndarray((NGROUP,NWAVE,NBIN),float)
asym = np.ndarray((NGROUP,NWAVE,NBIN),float)
qsca = np.ndarray((NGROUP,NWAVE,NBIN),float)


for ifile in range(1):
    f1 = open(direc +cases[ifile] + r'/run/carma/carma_nuc_grow_mie.txt', 'r')
    infoLine = f1.readline()

    for iw in range(NWAVE):
        wInfo = f1.readline()
        wInfo = wInfo.strip().split()
        wave[iw] = wInfo[1] ## unit [cm]

    for ig in range(NGROUP):
        for ibin in range(NBIN):
            rInfo = f1.readline()
        for iw in range(NWAVE):
            wInfo = f1.readline()
            wInfo = wInfo.strip().split()
            r_refidx_ice[ ig,iw] = wInfo[1]
            i_refidx_ice[ ig,iw] = wInfo[2]
            r_refidx_water[ ig,iw] = wInfo[3]
            i_refidx_water[ ig,iw] = wInfo[4]

    for iit in range(200):
        info = f1.readline()
        for ig in range(NGROUP):
            for iw in range(NWAVE):
                for ibin in range(NBIN):
                    info = f1.readline()
#                     print(info)
                    info = info.strip().split()
                    qext[ ig,iw,ibin] = info[2]
                    ssa[ ig,iw,ibin] = info[3]
                    asym[ ig,iw,ibin] = info[4]

    f1.close()

qsca = qext * ssa



Reading carma_nuc_grow_mie.txt


In [6]:
temp_psg = np.array([300.7 , 299.38, 298.06, 296.82, 295.89, 295.77, 295.26, 294.2 ,
       293.58, 293.01, 292.33, 291.59, 290.71, 289.57, 288.28, 286.86,
       285.28, 283.2 , 280.4 , 278.04, 275.2 , 273.19, 269.16, 266.8 ,
       263.8 , 258.74, 254.58, 247.18, 238.11, 229.71, 219.37, 210.18,
       202.64, 197.32, 193.29, 191.27, 192.62, 198.59, 204.41, 205.33,
       209.02, 213.9 , 216.71, 217.83, 220.37, 220.92, 227.81, 232.43,
       238.87, 248.74, 247.93, 248.22, 249.41, 252.28, 256.31, 259.73,
       262.  , 263.78, 264.76, 262.57, 256.7 , 249.89, 242.81, 236.74,
       232.21, 226.42, 218.09, 209.2 , 200.09, 190.96, 182.55, 184.98])
pres_psg = np.array([1.0028e+00, 9.8757e-01, 9.7239e-01, 9.5712e-01, 9.4192e-01,
       9.2673e-01, 9.1152e-01, 8.9630e-01, 8.8107e-01, 8.6585e-01,
       8.5062e-01, 8.3538e-01, 8.1761e-01, 7.9483e-01, 7.6946e-01,
       7.4408e-01, 7.1872e-01, 6.9335e-01, 6.6165e-01, 6.2363e-01,
       5.8559e-01, 5.4756e-01, 5.0956e-01, 4.7156e-01, 4.3358e-01,
       3.9560e-01, 3.5765e-01, 3.1372e-01, 2.6695e-01, 2.2670e-01,
       1.9258e-01, 1.6366e-01, 1.3912e-01, 1.1825e-01, 1.0051e-01,
       8.5439e-02, 7.2558e-02, 6.1496e-02, 5.2016e-02, 4.3910e-02,
       3.6993e-02, 3.1089e-02, 2.6049e-02, 2.1761e-02, 1.8124e-02,
       1.5050e-02, 1.2460e-02, 1.0285e-02, 8.4564e-03, 6.9183e-03,
       5.6318e-03, 4.5617e-03, 3.6765e-03, 2.9483e-03, 2.3526e-03,
       1.8679e-03, 1.4757e-03, 1.1600e-03, 9.0729e-04, 7.0596e-04,
       5.4629e-04, 4.2042e-04, 3.2178e-04, 2.4494e-04, 1.8542e-04,
       1.3960e-04, 1.0452e-04, 7.7673e-05, 5.6793e-05, 4.0142e-05,
       2.6350e-05, 1.5000e-05])

pres_psg = pres_psg * 1e5

from scipy import interpolate

fice = interpolate.interp1d(pres_psg,temp_psg,kind='linear')

temp_p = fice([10000])
# temp_p[0]

In [7]:
## WITHOUT INTERPOLATION

qext_w = np.zeros(NWAVE,float)
qsca_w = np.zeros(NWAVE,float)
omega_w = np.zeros(NWAVE,float)
g_w = np.zeros(NWAVE,float)
# x = z[0][:]

ifile = 0
# it = -1

igroup = 2 #ice cloud # 3 for liquid cloud
ielem = 2 #Ice # 4 for Liquid cloud
ibin = 33 # 50 um for Ice     26 for Liq
mmr = 1e-7 # for ice

# for i, height in enumerate(eff_layers):
for ip in range(10):
#     iz = ih * 7
    f = open(f"/project2/abbot/jeffyang/PSG/UnitCldSpec_Height/cloudspec_ice_{ip}.txt","w")
    conc = mmr * (ip*10000+5000) / fice([ip*10000+5000])[0] /297000. / rho[ielem] /(4/3 * np.pi *(r[ifile,igroup,ibin]/1e4)**3)
    print("Ice Conc = ",conc)

    totalmass = rmass[ifile,igroup,ibin] / 1000. * conc
    print("Ice Mass = ",totalmass)
    
    for iw in range(NWAVE):
        
        qsca_w[iw] = qsca[igroup,iw,ibin] * np.pi * (r[ifile,igroup,ibin]/1e6)**2 * conc
        qext_w[iw] = qext[igroup,iw,ibin] * np.pi * (r[ifile,igroup,ibin]/1e6)**2 * conc
        g_w[iw] = asym[igroup,iw,ibin] * ssa[igroup,iw,ibin]
        
#             igroup = 3 #liquid cloud
#             ielem = 4 
#             for ibin in range(NBIN):
#                 qsca_w[iw] = qsca_w[iw] + qsca[igroup,iw,ibin] * np.pi * (r[ifile,igroup,ibin]/1e6)**2 * conc_all[ifi,it,ielem,iz,ibin]
#                 qext_w[iw] = qext_w[iw] + qext[igroup,iw,ibin] * np.pi * (r[ifile,igroup,ibin]/1e6)**2 * conc_all[ifi,it,ielem,iz,ibin]
#                 g_w[iw] = g_w[iw] + asym[igroup,iw,ibin] * ssa[igroup,iw,ibin]
#                 totalmass = totalmass + rmass[ifile,igroup,ibin] / 1000. * conc_all[ifi,it,ielem,iz,ibin]
            
        qext_newdtype = qext_w[iw].astype(np.float128)
        qsca_newdtype = qsca_w[iw].astype(np.float128)
        qext_newdtype = qext_newdtype / totalmass
        qsca_newdtype = qsca_newdtype / totalmass
        omega_w[iw] = qsca_newdtype / qext_newdtype
        g_w[iw] = g_w[iw] / (ssa[igroup,iw,:].sum())

        lines = " ".join(["{:.4e}".format(wave[iw] * 1e4),"{:.4e}".format(qext_newdtype),"{:.4e}".format(omega_w[iw]),"{:.4e}".format(g_w[iw])]) + "\n"
    #     lines = [wave[iw],qext_w[iw],omega_w[iw],g_w[iw]]
        f.writelines(lines)
    #     f.writelines(["{:.2e}".format(wave[iw]),"{:.2e}".format(qext_w[iw]),"{:.2e}".format(omega_w[iw]),"{:.2e}".format(g_w[iw])])
    #     f.writelines(['\n'])
    f.close()
    

Ice Conc =  1.4741016154617129e-05
Ice Mass =  8.490825305059465e-15
Ice Conc =  4.39344778600603e-05
Ice Mass =  2.530625924739473e-14
Ice Conc =  6.429957861748187e-05
Ice Mass =  3.7036557283669554e-14
Ice Conc =  8.336679126001477e-05
Ice Mass =  4.801927176576851e-14
Ice Conc =  0.00010241253568161907
Ice Mass =  5.898962055261258e-14
Ice Conc =  0.00012140549489902085
Ice Mass =  6.9929565061836e-14
Ice Conc =  0.000140217508614407
Ice Mass =  8.076528496189844e-14
Ice Conc =  0.00015755620752275974
Ice Mass =  9.075237553310961e-14
Ice Conc =  0.0001754428534380173
Ice Mass =  1.0105508358029795e-13
Ice Conc =  0.00019338096642480566
Ice Mass =  1.1138743666068805e-13


In [12]:
## CARMA Cloud

#generates final input files
LA_nir = open("/project2/abbot/jeffyang/PSG/PSG_input/basefiles/psg_test_input_nir.txt", "r")
LA_uv = open("/project2/abbot/jeffyang/PSG/PSG_input/basefiles/psg_test_input_uv.txt", "r")
LA_vis = open("/project2/abbot/jeffyang/PSG/PSG_input/basefiles/psg_test_input_vis.txt", "r")
rantypes = [LA_nir, LA_uv, LA_vis]
types = ['nir', 'uv', 'vis']

# template_spec = []
# for nth in range(10):
#     template_spec.append('0')
for ip in range(10):

    for k, rantype in enumerate(rantypes):
        type_name = types[k]
        rantype.seek(0)
        lines = rantype.readlines()
        for line_num, line in enumerate(lines):
                
            if "<ATMOSPHERE-LAYERS-MOLECULES>" in line:
                lines[line_num] = lines[line_num].replace("\n",",") 
                lines[line_num] = lines[line_num] + "UnitCld" + ","
                lines[line_num] = (lines[line_num])[:-1]
                lines[line_num] = lines[line_num] + "\n"
                
            if "<ATMOSPHERE-LAYER-1>" in line:
                for height in range(72):
                    lst_temp = lines[line_num + height].replace(f"<ATMOSPHERE-LAYER-{height + 1}>", '') #EDITED; replaced old version of ".removeprefix(f"<ATMOSPHERE-LAYER-{height + 1}>")"
                    lst_temp1 = lst_temp.split(',')
                    pressure = float(lst_temp1[0]) * 1e5

                    add_spec = "{:.4e}".format(1e-7)
                    lines[line_num + height] = lines[line_num + height].replace("\n", ",")
                    if (pressure > ip*10000) and (pressure < ip*10000+10000):
                        add_spec = "{:.4e}".format(1e-7)
                        lines[line_num + height] = lines[line_num + height] + add_spec + "\n"
                    else:
                        add_spec = "{:.4e}".format(0)
                        lines[line_num + height] = lines[line_num + height] + add_spec + "\n"

        
            if "<ATMOSPHERE-NAERO>" in line:
                lines[line_num] = lines[line_num].replace("0\n", "20\n")
                
            if "<ATMOSPHERE-NMAX>" in line:
                lines[line_num] = lines[line_num].replace("0\n", "1\n")
            
            if "<ATMOSPHERE-LMAX>" in line:
                lines[line_num] = lines[line_num].replace("0\n", "80\n")
                
            if "<ATMOSPHERE-AEROS>" in line:
                lines[line_num] = "<ATMOSPHERE-AEROS>"
                lines[line_num] =  lines[line_num] + "UnitCld" + ","
                lines[line_num] = (lines[line_num])[:-1]
                lines[line_num] = lines[line_num] + "\n"
                    
            if "<ATMOSPHERE-ATYPE>" in line:
                lines[line_num] = "<ATMOSPHERE-ATYPE>"
                lines[line_num] = lines[line_num] + "UnitCld" + "_T5_USER" + ","
                lines[line_num] = (lines[line_num])[:-1]
                lines[line_num] = lines[line_num] + "\n"
            
            if "<ATMOSPHERE-AABUN>" in line:
                lines[line_num] = "<ATMOSPHERE-AABUN>"
                lines[line_num] = lines[line_num] + "1" + ","
                lines[line_num] = (lines[line_num])[:-1]
                lines[line_num] = lines[line_num] + "\n"
            
            if "<ATMOSPHERE-AUNIT>" in line:
                lines[line_num] = "<ATMOSPHERE-AUNIT>"
                lines[line_num] = lines[line_num] + "scl" + ","
                lines[line_num] = (lines[line_num])[:-1]
                lines[line_num] = lines[line_num] + "\n"
                    
            if "<ATMOSPHERE-ASIZE>" in line:
                lines[line_num] = "<ATMOSPHERE-ASIZE>"
                lines[line_num] = lines[line_num] + "0" + ","
                lines[line_num] = (lines[line_num])[:-1]
                lines[line_num] = lines[line_num] + "\n"
            
            if "<ATMOSPHERE-ASUNI>" in line:
                lines[line_num] = "<ATMOSPHERE-ASUNI>"
                lines[line_num] = lines[line_num] + "um" + ","
                lines[line_num] = (lines[line_num])[:-1]
                lines[line_num] = lines[line_num] + "\n"         

        fadd = open(f"/project2/abbot/jeffyang/PSG/UnitCldSpec_Height/cloudspec_ice_{ip}.txt","r")
        fadd.seek(0)
        lines_temp = fadd.readlines()
        lines.append(f"<SPEC>UnitCld_T5_USER\n")
        lines = lines + lines_temp
        lines.append("</SPEC>\n")
        
        augmented_input_file = open(f"/project2/abbot/jeffyang/PSG/PSG_input/SensiTest/HeightTest/input_ice_{ip}_{type_name}.txt","w")
        augmented_input_file.writelines(lines)
        augmented_input_file.close()             
